## Generate Logical Rule VQA Pipeline - T11

In [1]:
import numpy as np
import pandas as pd
import random
import ast
from pprint import pprint
from collections import Counter
import re
import os
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

## Question Logic_rule Template 11

### Questions and Blender generation

In [2]:
## Question Logic_rule Template 2
lr_ques_t11 = pd.read_csv("/home/jingying/git_paper_1/diverse_questions/lr_questions_rose/questions_lr_t11/question_results_11.csv")
print(lr_ques_t11.shape)

# Drop duplicates based on Question. 
lr_ques_t11 = lr_ques_t11.drop_duplicates(subset=['Question'])
print(lr_ques_t11.shape)

(2376, 6)
(2376, 6)


In [3]:
# Filter Allergy Fruit where 'fa' is not 'None'
df = lr_ques_t11
lr_t11_ques_values = df[df['fa'] != 'None']
lr_t11_ques_none = df[df['fa'] == 'None']

print(lr_t11_ques_values.shape)
print(lr_t11_ques_none.shape)

(1179, 6)
(1197, 6)


In [4]:
# Function to randomly select 10 items
def random_select_8(items):
    item_list = items.split(', ')
    if len(item_list) > 10:
        return ', '.join(random.sample(item_list, 10)) 
    return ', '.join(item_list) 


# Function to split 'fa' values into two halves
def split_fa_values(row):
    fa_values = row['answers'].split(', ')
    random.shuffle(fa_values)  
    mid_point = (len(fa_values) + 1) // 2  
    ground_truth_ans = ', '.join(fa_values[:mid_point]) 
    spatial_exclude_ans = ', '.join(fa_values[mid_point:])  
    return ground_truth_ans, spatial_exclude_ans


# Generate Blender Visible Data

# Define blend_middle and blend_noise sets
blend_middle = set(['bread', 'egg', 'meat', 'wine', 'chocolate', 'candy'])

blend_noise = set([
'bottle','lemon', 'orange', 'grape', 'banana',  
'cheese', 'onion', 'tomato', 'carrot', 'eggplant', 'pumpkin', 'mushroom',
'cucumber', 'pepper', 'garlic', 'watermelon', 'avocado', 'potato', 'plum', 
'lime', 'yogurt', 'milk', 'asparagus', 'broccoli', 'leek', 
'capsicum', 'chilli', 'zucchini','cabbage', 'lettuce', 'springonion', 'parsley', 'pear', 'apple' ])

def add_objects(row):
    fa_values = row['fa']
    if fa_values == 'None':
        fa_set = set()
    else:
        fa_set = set(fa_values.split(', '))
    
    ground_truth_ans_set = set(row['ground_truth_ans'].split(', '))
    spatial_exclude_ans_set = set(row['spatial_exclude_ans'].split(', '))
    
    # Remove 'None' from sets
    ground_truth_ans_set.discard('None')
    
    # For available noise objects
    available_noise = list(blend_noise - fa_set)
    random.shuffle(available_noise)
    
    # Ground_Truth: For Right_side_object
    right_side_object = list(ground_truth_ans_set) + available_noise[:max(0, 5 - len(ground_truth_ans_set))]
    right_side_object_str = ', '.join(right_side_object).lstrip(', ')

    # Noise_Truth: For Left_side_object
    left_side_object = list(spatial_exclude_ans_set) + available_noise[:max(0, 5 - len(spatial_exclude_ans_set))]
    left_side_object_str = ', '.join(left_side_object).lstrip(', ')
    
    # For middle_object
    middle_object = random.sample(blend_middle, 1)[0]

    return right_side_object_str, left_side_object_str, middle_object

def process_data(df, filename):
    
    full_output_path = f"{base_path}{filename}_visible_full.csv"
    blend_output_path = f"{base_path}{filename}_blend.csv"

    # Apply the functions
    df['answers'] = df['fa'].apply(random_select_8)
    df[['ground_truth_ans', 'spatial_exclude_ans']] = df.apply(split_fa_values, axis=1, result_type='expand')
    df[['right_side_object', 'left_side_object', 'middle_object']] = df.apply(add_objects, axis=1, result_type='expand')
    blend_df_t10 = df[['right_side_object', 'left_side_object', 'middle_object']]

    df.to_csv(full_output_path)
    blend_df_t10.to_csv(blend_output_path)

base_path = "/home/jingying/git_paper_1/diverse_questions/lr_questions_rose/questions_lr_t11/"

process_data(lr_t11_ques_values, "lr_ques_t11")
print(lr_t11_ques_values.shape)

(1179, 12)


### Read the generated files

In [2]:
lr_ques_t11 = pd.read_csv("/home/jingying/git_paper_1/diverse_questions/lr_questions_rose/questions_lr_t11/lr_ques_t11_visible_full.csv")
lr_t11_col = lr_ques_t11[['Question', 'fa', 'answers', 'ground_truth_ans', 'spatial_exclude_ans', 'right_side_object', 'left_side_object', 'middle_object']]
blend_df_t11 = pd.read_csv("/home/jingying/git_paper_1/diverse_questions/lr_questions_rose/questions_lr_t11/lr_ques_t11_blend.csv")
print(lr_ques_t11.shape)

(1179, 13)


In [12]:
lr_t11_col[:2]

,Question,fa,answers,ground_truth_ans,spatial_exclude_ans,right_side_object,left_side_object,middle_object
0,"Which Food options would you suggest with either edible fungi or having at least three nutrients, and also are not bitter?","orange, carrot, cucumber, potato, pumpkin, avocado, mushroom, cabbage, broccoli, asparagus, onion, tomato, apple, banana, lemon, plum, lime, watermelon, pear, grape, yogurt, cheese, milk","pear, plum, onion, cheese, cucumber, yogurt, grape, mushroom, orange, broccoli","broccoli, yogurt, cheese, grape, pear","orange, mushroom, onion, plum, cucumber","yogurt, grape, cheese, pear, broccoli","onion, cucumber, plum, mushroom, orange",chocolate
1,"If I want to avoid fishy foods, which items of Food are recommended that have at least three nutrients or have edible fungi?","orange, carrot, cucumber, potato, pumpkin, avocado, mushroom, cabbage, broccoli, asparagus, onion, tomato, apple, banana, lemon, plum, lime, watermelon, pear, grape, yogurt, cheese, milk","plum, cheese, broccoli, banana, onion, pear, pumpkin, cabbage, lime, cucumber","pear, broccoli, plum, onion, cabbage","pumpkin, banana, cucumber, lime, cheese","onion, plum, cabbage, pear, broccoli","cucumber, banana, cheese, pumpkin, lime",chocolate


### Add spatial info to questions and in questions_spatial column 

For images: 0-500
- lr_t11: add 'in front of the leopard, but not those inside and around the bag, '
- lr_t11: add 'between the middle object and the toy leopard but excluding those around the bag, ' 
- lr_t11: add 'positioned further left than the leopard in the image, ' 


For images: 500-1179
- lr_t11: add 'on the left side of the leopard, which is on the table, in the image from the leopard's perspective, '
- lr_t11: add ', among the six closest to the front edge of the table, '
- lr_t11: add 'to the right side of the middle object in the image '
- lr_t11: add 'in the lower half of the image'

In [3]:
lr_ques_t11 = pd.read_csv("/home/jingying/git_paper_1/diverse_questions/lr_questions_rose/questions_lr_t11/lr_ques_t11_visible_full.csv")
print(lr_ques_t11.shape)

(1179, 13)


In [4]:
lr_t11_col[:2]

,Question,fa,answers,ground_truth_ans,spatial_exclude_ans,right_side_object,left_side_object,middle_object
0,"Which Food options would you suggest with either edible fungi or having at least three nutrients, and also are not bitter?","orange, carrot, cucumber, potato, pumpkin, avocado, mushroom, cabbage, broccoli, asparagus, onion, tomato, apple, banana, lemon, plum, lime, watermelon, pear, grape, yogurt, cheese, milk","pear, plum, onion, cheese, cucumber, yogurt, grape, mushroom, orange, broccoli","broccoli, yogurt, cheese, grape, pear","orange, mushroom, onion, plum, cucumber","yogurt, grape, cheese, pear, broccoli","onion, cucumber, plum, mushroom, orange",chocolate
1,"If I want to avoid fishy foods, which items of Food are recommended that have at least three nutrients or have edible fungi?","orange, carrot, cucumber, potato, pumpkin, avocado, mushroom, cabbage, broccoli, asparagus, onion, tomato, apple, banana, lemon, plum, lime, watermelon, pear, grape, yogurt, cheese, milk","plum, cheese, broccoli, banana, onion, pear, pumpkin, cabbage, lime, cucumber","pear, broccoli, plum, onion, cabbage","pumpkin, banana, cucumber, lime, cheese","onion, plum, cabbage, pear, broccoli","cucumber, banana, cheese, pumpkin, lime",chocolate


In [5]:
# Define two sets of spatial information options
spatial_info_options_0_500 = [
    'in front of the leopard, but not those inside and around the bag, ',
    'between the middle object and the toy leopard but excluding those around the bag, ',
    'positioned further left than the leopard in the image, '
]

spatial_info_options_500_end = [
    'on the left side of the leopard, which is on the table, in the image from the leopard\'s perspective, ',
    'among the six closest to the front edge of the table, ',
    'to the right side of the middle object in the image, ',
    'in the lower half of the image'
]

# Function to insert spatial information into the question
def insert_spatial_info(question, middle_object, index):
    # Choose the appropriate set of spatial information based on the index
    if index < 500:
        spatial_info = [info.replace("middle object", middle_object) for info in spatial_info_options_0_500]
    else:
        spatial_info = [info.replace("middle object", middle_object) for info in spatial_info_options_500_end]

    # Randomly choose one spatial information
    chosen_spatial_info = random.choice(spatial_info)

    # Convert words to lowercase and change 'with' to 'have'
    question = re.sub(r'\b(Food|Vegetable|Fruit|Meat|Dairy|Grain|Seed|Fish)\b', lambda match: match.group(0).lower(), question)

    # Identify key phrases to target
    key_phrases = ["category", "options", 
                   "which items of dairy", "which items of fish", 
                   "which items of food", "which items of fruit", 
                   "which items of grain", "which items of meat", 
                   "which items of seed", "which items of vegetable"]

    # Insert spatial information after the identified phrase
    for phrase in key_phrases:
        if phrase in question.lower():
            split_point = question.lower().find(phrase) + len(phrase)
            return question[:split_point] + " " + chosen_spatial_info + question[split_point:]

    return question 

df = lr_ques_t11.copy()
# Applying the function to your DataFrame
df['questions_spatial'] = df.apply(lambda row: insert_spatial_info(row['Question'], row['middle_object'], row.name), axis=1)
# df.to_csv("/home/jingying/git_paper_1/diverse_questions/lr_questions_rose/questions_lr_t11/lr_ques_t11_visible_full.csv")
print(df.shape)

(1179, 14)


In [13]:
# Add logical_operator, logical_level, logical_types
def add_logical_columns(df):

    df['logical_operator'] = 'implication'
    df['logical_level'] = 'level 3'
    df['logical_types'] = 'conditional'
    return df

df = add_logical_columns(df)

df.to_csv("/home/jingying/git_paper_1/diverse_questions/lr_questions_rose/questions_lr_t11/lr_ques_t11_visible_full.csv")
print(df.shape)

(1179, 17)


In [14]:
def replace_underscores(df):

    df['Question'] = df['Question'].str.replace('_', ' ', regex=False)
    df['questions_spatial'] = df['questions_spatial'].str.replace('_', ' ', regex=False)
    return df

df = replace_underscores(df)
df.to_csv("/home/jingying/git_paper_1/diverse_questions/lr_questions_rose/questions_lr_t11/lr_ques_t11_visible_full.csv")
print(df.shape)

(1179, 17)


In [16]:
lr_ques_t11 = pd.read_csv("/home/jingying/git_paper_1/diverse_questions/lr_questions_rose/questions_lr_t11/lr_ques_t11_visible_full.csv")
print(lr_ques_t11.shape)

(1179, 18)


### Check blender objects

In [ ]:
#### Check blender objects

blend_onto = set([
    'bottle','plant', 'lemon', 'orange', 'grape', 'banana', 'pan', 'pot', 
    'cheese', 'onion', 'candy', 'tomato', 'carrot', 'eggplant', 'pumpkin', 'mushroom',
    'cucumber', 'pepper', 'garlic', 'watermelon', 'avocado', 'wine', 'potato', 'plum', 
    'lime', 'chocolate', 'yogurt', 'milk', 'asparagus', 'broccoli', 'leek', 
    'capsicum', 'chilli', 'zucchini','cabbage', 'lettuce', 'springonion', 'parsley', 'pear',
    'tomatonew', 'GreenPear', 'bread', 'egg', 'meat', 'apple' 
])

# unique values in lr_ques_t1_values 
df = lr_t7_col.copy()
all_items = df['fa'].str.split(', ').explode()
unique_items = all_items.unique()
unique_items = list(unique_items)

print(unique_items)

# Sample lists
list1 = unique_items
list2 = blend_onto

# Finding values in list1 not in list2 and vice versa
different_values = set(list1).symmetric_difference(set(list2))

# Converting the set to a list for easier usage
different_values_list = list(different_values)

print(different_values_list)


### Reference

In [9]:
# Unique answers distribution

def analyze_unique_combinations(df, column_name):

    df_filtered = df[df[column_name].notna() & (df[column_name] != '')]

    df_filtered['normalized'] = df_filtered[column_name].apply(lambda x: ','.join(sorted(x.split(','))))

    unique_combinations = Counter(df_filtered['normalized'])
    num_rows_with_values = len(df_filtered)
    num_unique_combinations = len(unique_combinations)

    return num_rows_with_values, num_unique_combinations, unique_combinations


num_values, num_combinations, combinations = analyze_unique_combinations(lr_t11_col, 'ground_truth_ans')


print(f"Number of rows with values: {num_values}")
print(f"Number of unique combinations: {num_combinations}")
print("Unique combinations and their counts:")
for combination, count in combinations.items():
    print(f"{combination}: {count}")

Number of rows with values: 1179
Number of unique combinations: 827
Unique combinations and their counts:
 cheese, grape, pear, yogurt,broccoli: 1
 broccoli, cabbage, onion, plum,pear: 1
 cabbage, cucumber, milk, tomato,avocado: 1
 asparagus, cheese, mushroom, watermelon,plum: 1
 apple, cucumber, grape, plum,onion: 1
 banana, cabbage, onion, potato,yogurt: 1
 avocado, cabbage, cucumber,mushroom: 1
 apple, asparagus, cucumber, mushroom,banana: 1
 apple, avocado, cabbage, mushroom,carrot: 1
 apple, avocado, broccoli, carrot,lemon: 1
 asparagus, lime, orange, yogurt,carrot: 1
 cabbage, cucumber, pear, tomato,broccoli: 1
 lime, onion, orange, plum,lemon: 1
 asparagus, avocado, broccoli, grape,yogurt: 1
 mushroom, orange, potato, pumpkin,plum: 1
 asparagus, banana, cabbage, pumpkin,milk: 1
 avocado, lime, mushroom, orange,grape: 1
 banana, grape, lemon, plum,lime: 1
 apple, banana, cheese, watermelon,yogurt: 1
 cabbage, lemon, potato, yogurt,cucumber: 1
 banana, cucumber, lime, tomato,yogur